In [1]:
import torch
import wandb
import numpy as np
from tqdm import trange
from src.models.base_texture_nca import BaseTextureNCA
from src.loss.vgg_loss import VGGLoss
from src.loss.mse_loss import MSELoss
from src.trainer.nca_trainer import NCATrainer
from src.utils.utils import *
from src.utils.EarlyStopping import EarlyStopping

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

wandb.login()

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from C:\Users\GAI\_netrc.
wandb: Currently logged in as: apc582nntscott (apc582nntscott-nycu-gia-ail) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# config

In [2]:
# 1. 設定預設 Config
default_config = {
    # --- 實驗管理 (WandB) ---
    'project_name': 'base_texture_nca',   # WandB 專案名稱 (所有實驗會歸類在此專案下)
    'run_name': 'test_vgg_baseline',      # [重要] 實驗名稱，建議包含關鍵變數 (如 'texture_vgg_ch16')
    'log_interval': 100,                  # 每隔多少步 (Steps) 記錄一次圖片到 WandB
    'seed': 42,                           # 隨機與 Numpy 種子，固定此值可確保每次實驗結果一致 (Reproducibility)

    # --- 資料與目標 ---
    'target_img_path': r'C:\Users\GAI\Desktop\Scott\NCA_Research\dataset\dotted_0201.jpg', # 目標紋理圖片路徑 (建議用 raw string r'...')
    'img_size': 128,                      # 訓練與生成的圖片解析度 (H=W=128)，NCA 具有縮放不變性，此值影響訓練顯存與視野
    'loss_type': 'vgg',                   # [重要] 損失函數類型：
                                          # 'mse': 像素級對齊 (生成結果會模糊，位置固定)
                                          # 'vgg': 風格/紋理對齊 (生成結果清晰，允許位置隨機，推薦用於紋理)

    # --- 模型架構 (NCA Model) ---
    'chn': 12,                            # NCA 狀態總通道數 (State Channels)。
                                          # 通常前 3 ch 為 RGB 顏色，後 9 ch 為隱藏狀態 (Hidden States)
    'hidden_n': 96,                       # 感知器 (MLP) 中間層的神經元數量。越大模型越強，但運算越慢 (通常 64~128)
    'use_gradient_checkpoint': False,     # 是否開啟梯度檢查點 (Gradient Checkpointing)。
                                          # False: 速度快，顯存佔用大
                                          # True: 速度慢 (約慢 30%)，但極度節省顯存 (可跑大 Batch 或大圖)

    # --- NCA 動力學 (Dynamics) ---
    'step_min': 32,                       # 訓練時，NCA 最小演化步數
    'step_max': 96,                       # 訓練時，NCA 最大演化步數
                                          # (隨機在 [min, max] 間取值，讓模型學會「長短期」都能維持圖案穩定)

    # --- 訓練策略 (Training Loop) ---
    'batch_size': 8,                      # 每次訓練抓取幾張圖進行更新 (顯存允許下越大越好，通常 4~16)
    'lr': 2e-3,                           # 學習率 (Learning Rate)，NCA 較敏感，建議 1e-3 ~ 2e-3
    'max_steps': 5000,                    # 總訓練步數，通常 2000 步可見雛形，5000~10000 步收斂
    
    # --- 樣本池 (Sample Pool) ---
    'pool_size': 1024,                    # 經驗回放池 (Replay Pool) 的大小。越大能記住越多狀態，訓練越穩定
    'pool_reset_freq': 8,                 # 每隔幾步 (Global Steps)，強制重置 Batch 中的一個樣本為初始種子。
                                          # 作用：防止模型忘記「如何從頭生長」，確保持續生長能力 (Regeneration)
}

# define logic

## setup case

In [3]:
def setup_case(config):
    """初始化所有訓練所需物件"""
    # 1. 設定種子
    set_seed(config.seed)
    
    # 2. 準備目標圖片 & Loss
    # 假設你有一個 target.jpg，若無可先用雜訊代替測試
    try:
        target_img = load_target_image(config.target_img_path, config.img_size, device)
    except FileNotFoundError:
        print(f"Warning: {config.target_img_path} not found. Using random target.")
        target_img = torch.rand(1, 3, config.img_size, config.img_size).to(device)
    
    # 3. 依照 config 選擇 Loss 函數
    if config.loss_type == 'mse':
        print("Using Simple MSE Loss")
        loss_fn = MSELoss(target_img)
    else:
        print("Using VGG / OT Loss")
        loss_fn = VGGLoss(target_img, device=device)

    # 4. 建立模型
    model = BaseTextureNCA(config).to(device)
    
    # 5. 建立優化器與排程器
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    # 範例 Scheduler: 2000 步後降低學習率
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2000], gamma=0.3)
    
    # 6. 建立資料池 (Data Pool)
    # 直接使用 Model 的 seed 方法產生初始池
    pool = model.seed(config.pool_size, sz=config.img_size)
    
    return model, optimizer, scheduler, pool, loss_fn

## train

In [4]:
def train(model, optimizer, scheduler, pool, loss_fn, config):
    """
    訓練迴圈 (Refactored)。
    包含 Config 驅動的步數、早停機制與模組化 Logging。
    """
    
    # 1. 初始化 Trainer
    trainer = NCATrainer(
        model=model,
        pool=pool,
        optimizer=optimizer,
        scheduler=scheduler,
        loss_fn=loss_fn,
        config=config,
        device=device
    )

    # 2. 從 Config 讀取訓練參數 (設定預設值以防漏填)
    max_steps = config.get('max_steps', 5000)
    log_interval = config.get('log_interval', 100)
    
    # 3. 初始化早停機制
    # NCA 訓練通常會有波動，建議 patience 設大一點 (例如 500-1000)
    early_stopper = EarlyStopping(
        patience=config.get('early_stop_patience', 1000),
        min_delta=config.get('early_stop_min_delta', 1e-5)
    )

    # 4. 開始訓練迴圈
    # 使用 trange 顯示進度條
    progress_bar = trange(max_steps, desc="Training")
    
    for i in progress_bar:
        # --- 訓練一步 ---
        stats = trainer.train_step(current_step=i)
        train_loss = stats['loss']
        
        # --- 準備 Log 資訊 ---
        log_dict = {
            "train/loss": train_loss,
            "train/lr": stats['lr'],
            "step": i
        }

        # --- 視覺化 (使用 Utils) ---
        if i % log_interval == 0:
            log_dict["train/image"] = log_batch_image(stats['batch_x'], step=i)

        # --- Wandb Log ---
        wandb.log(log_dict)
        
        # --- 更新進度條文字 ---
        if i % 10 == 0:
            progress_bar.set_description(f"Loss: {train_loss:.5f}")

        # --- 早停檢查 ---
        # 這裡示範基於 Train Loss 的早停 (若 Loss 長期卡住不動則停)
        if early_stopper(train_loss):
            print(f"\n[Early Stopping] Triggered at step {i}. Best Loss: {early_stopper.best_loss:.5f}")
            break

## test

In [5]:
def test(model, loss_fn, config):
    """
    測試/生成階段：產生演化影片，並記錄 Loss 曲線與執行早停。
    """
    print("Testing (Generating Video & Monitoring Loss)...")
    
    # 1. 準備設定
    test_steps = config.get('test_steps', 300)      # 測試總步數
    patience = config.get('test_patience', 50)      # 測試階段的早停耐心值
    img_size = config.get('img_size', 128)
    
    # 2. 初始化早停器 (這裡設定 min_delta 稍小，目的是檢測收斂)
    stopper = EarlyStopping(patience=patience, min_delta=1e-5)
    
    model.eval() # 設定為評估模式
    
    with torch.no_grad():
        # 初始化種子
        x = model.seed(1, sz=img_size)
        
        frames = []
        test_losses = []
        
        iterator = trange(test_steps, desc="Generating")
        
        for i in iterator:
            # A. 模型推論
            x = model(x)
            
            # B. 計算 Loss (用於評估品質與穩定性)
            # 注意：NCA 輸出 x 是 [1, 12, H, W]，Loss 需要 RGB [1, 3, H, W]
            rgb_x = to_rgb(x)

            # 計算 Loss (MSE 或 VGG)
            # 因為 x 是 batch=1，直接取 item()
            current_loss = loss_fn(rgb_x).item()
            test_losses.append(current_loss)
            
            # C. 處理圖片 (存入 Frames)
            # [1, 3, H, W] -> [H, W, 3] -> Numpy
            img = rgb_x[0].permute(1, 2, 0).cpu().numpy()
            img = np.clip(img, 0, 1)
            frames.append((img * 255).astype(np.uint8))
            
            # D. 更新進度條資訊
            iterator.set_postfix(loss=current_loss)
            
            # E. 安全檢查：如果 Loss 變成 NaN 或無限大，立刻停止 (NCA 常見問題)
            if np.isnan(current_loss) or np.isinf(current_loss):
                print(f"⚠️ Test stopped early due to instability (NaN/Inf) at step {i}")
                break

            # F. 早停機制：如果圖片已經穩定不再變化 (Loss 收斂)
            if stopper(current_loss):
                print(f"✅ Test stopped early (Converged) at step {i}. Best Loss: {stopper.best_loss:.5f}")
                break
                
        # 3. 記錄到 WandB
        # 轉換影片格式 [T, H, W, C] -> [T, C, H, W]
        frames = np.array(frames).transpose(0, 3, 1, 2)
        
        # 建立 Log Dictionary
        log_dict = {
            "test/video": wandb.Video(frames, fps=30, format="mp4"),
            "test/final_loss": test_losses[-1] if test_losses else 0,
            "test/steps_taken": len(frames)
        }
        
        # 繪製 Loss 曲線 (可以看穩定性)
        data = [[s, l] for s, l in enumerate(test_losses)]
        table = wandb.Table(data=data, columns=["step", "loss"])
        log_dict["test/loss_curve"] = wandb.plot.line(
            table, "step", "loss", title="Test Generation Stability"
        )
        
        wandb.log(log_dict)
        
    model.train() # 恢復訓練模式 (如果是 pipeline 中途測試的話)

## pipeline

In [6]:
def model_pipeline(hyperparams):
    """主流程"""
    # 這裡傳入 config=hyperparams 讓 wandb 幫我們管理 config
    full_run_name = get_next_run_name(
        project_name=hyperparams['project_name'], 
        base_name=hyperparams.get('run_name', None)
    )
    with wandb.init(project=hyperparams['project_name'], config=hyperparams, name=full_run_name):
        # 使用 wandb.config (這樣如果用 Sweeps 調整參數會自動生效)
        config = wandb.config
        
        # 1. Setup
        model, optim, sched, pool, loss_fn = setup_case(config)
        
        # 2. Train
        train(model, optim, sched, pool, loss_fn, config)
        
        # 3. Test
        test(model,loss_fn, config)
        
    return model

# run case

In [7]:
final_model = model_pipeline(default_config)

Using VGG / OT Loss


Loss: 208565.81250:  95%|█████████▌| 4758/5000 [22:50<01:09,  3.47it/s]                



[Early Stopping] Triggered at step 4758. Best Loss: 173627.48438
Testing (Generating Video & Monitoring Loss)...


Generating:  17%|█▋        | 51/300 [00:00<00:02, 95.46it/s, loss=2.76e+6]


✅ Test stopped early (Converged) at step 51. Best Loss: 1929846.00000


step,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇██
test/final_loss,▁
test/steps_taken,▁
train/loss,█▅▃▂▃█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,███████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,4758
test/final_loss,2760636.25
test/steps_taken,52
train/loss,230846.45312
train/lr,0.0006
